In [48]:
import pymysql
import pandas as pd
from datetime import datetime, timedelta

### googletrend 2008.07~현재까지의 경제keyword에 대한 검색량 counts정보 dataframe화
#### 1차목표: 2018년1월1일부터의 데이터만 저장.

In [147]:
googletrends_df = pd.read_csv('./data/18.01.01-20.10.17.csv', header=1, sep=',', skip_blank_lines = True)
googletrends_df = googletrends_df.rename({'주': 'date', '경제: (대한민국)':'ratio'}, axis = 'columns')
googletrends_df

,date,ratio
0,2017-12-31,54
1,2018-01-07,49
2,2018-01-14,57
3,2018-01-21,52
4,2018-01-28,57
...,...,...
141,2020-09-13,88
142,2020-09-20,92
143,2020-09-27,68
144,2020-10-04,86


In [148]:
"""
google: week를 sunday to saturday로 설정
--> 날짜를 하루씩 더함
"""
def plus_one_day(googletrends_df):
    date2 = []
    for i in range(len(googletrends_df)):
        after_one_day = datetime.strptime(googletrends_df.iloc[i].date, '%Y-%m-%d').date() + timedelta(days=1)
        after_one_day = after_one_day.strftime("%Y-%m-%d")
        date2.append(after_one_day)
    return date2

In [149]:
googletrends_df['date'] = plus_one_day(googletrends_df)
googletrends_df

,date,ratio
0,2018-01-01,54
1,2018-01-08,49
2,2018-01-15,57
3,2018-01-22,52
4,2018-01-29,57
...,...,...
141,2020-09-14,88
142,2020-09-21,92
143,2020-09-28,68
144,2020-10-05,86


### DB연동

In [150]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

In [151]:
cur.execute("show databases")
print(cur.fetchall())
cur.execute("use CLI")

(('information_schema',), ('CLI',), ('MusicBank',), ('mysql',), ('mytestdb',), ('performance_schema',), ('sys',))


0

### DB table 생성

In [152]:
"""
검색어 트랜드 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS google_counting(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    day bigint(2) NOT NULL,
    cnt double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)

0

In [153]:
cur.execute("desc google_counting")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,day,bigint(2),NO,,None,
4,cnt,double,YES,,None,


### google trend의 경제keyword counting정보 주별로 저장

In [154]:
"""
날짜에서 year, month, day 추출
"""
def extract_year_and_month_day(period):
    year = period.split('-')[0]
    month = period.split('-')[1]
    day = period.split('-')[2]
    return year, month, day


In [155]:
"""
db table에 저장
"""
for i in range(len(googletrends_df)):
    query = """ 
    Insert into google_counting (year, month, day, cnt) values (%d, %d, %d, %f) ;
    """
    year, month, day = extract_year_and_month_day(googletrends_df.iloc[i].date)
    google_cnt = googletrends_df.iloc[i].ratio

    mystring = (query % (int(year), int(month), int(day), float(google_cnt)))
    print(mystring)
    cur.execute(mystring)

 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 1, 1, 54.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 1, 8, 49.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 1, 15, 57.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 1, 22, 52.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 1, 29, 57.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 2, 5, 55.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 2, 12, 46.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 2, 19, 52.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 2, 26, 48.000000) ;
    
 
    Insert into google_counting2 (year, month, day, cnt) values (2018, 3, 5, 55.000000) ;
    
 
    Insert into google

In [156]:
# 확인
query = """ 
select * from google_counting ;
"""
cur.execute(query)
tmp=pd.DataFrame(list(cur.fetchall()))
tmp

,0,1,2,3,4
0,1,2018,1,1,54.0
1,2,2018,1,8,49.0
2,3,2018,1,15,57.0
3,4,2018,1,22,52.0
4,5,2018,1,29,57.0
...,...,...,...,...,...
141,142,2020,9,14,88.0
142,143,2020,9,21,92.0
143,144,2020,9,28,68.0
144,145,2020,10,5,86.0


In [157]:
conn.commit()
cur.close()
conn.close()